## Код, который не относится к проблеме переведен в `raw`, чтобы не запускаться.

In [1]:
%cd ../../..

/Users/b1/Desktop/m1p


In [2]:
from tqdm import tqdm
from typing import Dict
import time

import torch
import torch.fx
from torch.fx.node import Node

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import module2graph
from module2graph import GraphInterperterWithGamma
from fast_nas_adapt.src.resnet18 import ResNet18


Тут был код, который замерял время. Сейчас он в файле

In [3]:
class GraphInterperterWithGumbelSoftmaxGamma(GraphInterperterWithGamma):
    def __init__(self, mod, gamma_shift=0.0, temperature=1.0):
        self.gamma_shift = gamma_shift
        self.temperature = temperature
        super().__init__(mod)

    def init_gammas(self):
        i = 0
        gammas = []
        self.gammas_name = {}
        for node in self.graph.nodes:
            if node.op == 'call_module':
                gammas.append(np.random.randn()+self.gamma_shift)
                self.gammas_name[str(node)] = i# перевод в str тут для удобства. в реалньых методах это не нужно
                i+=1                        # да и вообще, тут по идее должен быть тензор/параметр
        self.gammas =  torch.nn.Parameter(torch.as_tensor(gammas), requires_grad = True)
        self.discrete = False 

    def sample_gammas(self):
        if self.discrete:
            return self.gammas
        else:
            d = torch.distributions.RelaxedBernoulli(logits=self.gammas, temperature=self.temperature)
            return d.rsample()
        
    def make_gammas_discrete(self):
        self.gammas.data = (self.gammas.data>=0) * 1.0
        self.gammas.requires_grad = False 
        self.discrete = True

Получим претрейн на n эпох

In [4]:
from fast_nas_adapt.src.cifar_data import get_dataloaders
from fast_nas_adapt.src import *

In [5]:
model = ResNet18(num_classes=10)#(num_classes=len(config.classes))


train_dl, test_dl = get_dataloaders(classes=range(10), batch_size=64,
                                    img_size=33)

Using cache found in /Users/b1/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/b1/Library/Python/3.8/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/b1/Library/Python/3.8/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified


Гиперпараметры:

In [6]:
imodel = GraphInterperterWithGumbelSoftmaxGamma(model.eval())

imodel = GraphInterperterWithGumbelSoftmaxGamma(model.eval(), 1.0)
optimizer = torch.optim.Adam([imodel.gammas])
loss_fn = torch.nn.CrossEntropyLoss()


In [7]:
### TEST
with torch.no_grad():
    x = torch.randn(64, 3, 32, 32)
    print(model(x)[0].shape, imodel(x).shape)

torch.Size([64, 10]) torch.Size([64, 10])


In [11]:
import time
ts = time.time()
ts

import datetime
datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d/%H:%M:%S')

'2023-03-12/19:25:03'

In [17]:
def get_timestamp():
    ts = time.time()
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d-%H:%M:%S')

In [19]:
epoch_number = 0

EPOCHS = 1

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print(f'EPOCH {epoch_number}:')

    imodel.train()

    epoch_history = []
    for i, (X, y) in tqdm(enumerate(train_dl), 'training', total=len(train_dl)):
        optimizer.zero_grad()
        y_pred = imodel(X)
        loss = loss_fn(y_pred, y)# + lambd * imodel.gammas.dot(times)
        loss.backward(retain_graph=True)
        optimizer.step()

        epoch_history.append(loss.item())
        
        if i == 10:
            break
    avg_loss = np.mean(epoch_history)

    imodel.eval()
    
    print('epoch trained')

    running_vloss = 0.0
    for i, (X, y) in tqdm(enumerate(test_dl), 'validating', total=len(test_dl)):
        if i == 10:
            break
        if X.shape[0] != 64:
            continue
        running_vloss += loss_fn(imodel(X), y)
    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_ts{}_epoch{}'.format(get_timestamp(), epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 0:


training:   1%|▍                               | 10/782 [00:20<25:47,  2.00s/it]


epoch trained


validating:   6%|█▉                            | 10/157 [00:02<00:32,  4.55it/s]


LOSS train 2.3232768232172187 valid 2.0951454639434814
